In [1]:
import torch

print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")


Torch version: 2.7.0+cu128
CUDA version: 12.8
CUDA available: True
GPU: NVIDIA GeForce RTX 3060


In [1]:
!pip install transformers datasets scikit-learn 


In [3]:
# Install dependencies (Run only once)
# pip install transformers datasets scikit-learn torch tqdm

# ----------------------
# Imports
# ----------------------
import torch
from torch.utils.data import DataLoader
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim import AdamW
from tqdm.auto import tqdm

# ----------------------
# Set device
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# ----------------------
# Load & Subsample Dataset
# ----------------------
dataset = load_dataset("amazon_polarity")

# Shuffle and subset
train_subset = dataset["train"].shuffle(seed=42).select(range(1_000_000))
test_subset  = dataset["test"].shuffle(seed=42).select(range(300_000))

# ----------------------
# Tokenization
# ----------------------
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_batch(batch):
    texts = [t + ". " + c for t, c in zip(batch["title"], batch["content"])]
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_subset = train_subset.map(tokenize_batch, batched=True, remove_columns=["title", "content"])
test_subset  = test_subset.map(tokenize_batch,  batched=True, remove_columns=["title", "content"])

train_subset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_subset.set_format("torch",  columns=["input_ids", "attention_mask", "label"])

train_ds = train_subset
val_ds   = test_subset

# ----------------------
# DataLoaders
# ----------------------
batch_size = 110  # adjust if memory is low
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)

# ----------------------
# Model, Optimizer, Scheduler
# ----------------------
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 2

total_steps = len(train_loader) * epochs  # epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10,
    num_training_steps=total_steps,
)

# ----------------------
# Training Loop
# ----------------------

for epoch in range(epochs):
    model.train()
    train_pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}")
    for batch in train_pbar:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        masks     = batch["attention_mask"].to(device)
        labels    = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_pbar.set_postfix(loss=loss.item())

    # ----------------------
    # Validation
    # ----------------------
    model.eval()
    all_preds, all_labels = [], []
    for batch in tqdm(val_loader, desc="Validation"):
        with torch.no_grad():
            logits = model(
                batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device)
            ).logits
        preds = logits.argmax(dim=-1).cpu().tolist()
        labels = batch["label"].cpu().tolist()
        all_preds.extend(preds)
        all_labels.extend(labels)

    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Epoch {epoch+1} ▶ Acc: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")


Using: cuda


Generating test split: 100%|██████████| 400000/400000 [00:00<00:00, 1025640.12 examples/s]
D:\AnacondaNav\AnacondaNavProgram\envs\SentimentAnalysis-bert\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bgari\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

Epoch 1 ▶ Acc: 0.9639 | Precision: 0.9748 | Recall: 0.9526 | F1: 0.9636


Validation: 100%|██████████| 2728/2728 [18:03<00:00,  2.52it/s]


Epoch 2 ▶ Acc: 0.9670 | Precision: 0.9663 | Recall: 0.9679 | F1: 0.9671


In [4]:
# ----------------------
# Save Model
# ----------------------
model.save_pretrained("saved_bert_sentiment_model")
tokenizer.save_pretrained("saved_bert_sentiment_model")
print("✅ Model saved to 'saved_bert_sentiment_model/'")


✅ Model saved to 'saved_bert_sentiment_model/'


In [5]:
# ----------------------
# Manual Input Prediction
# ----------------------
def predict_sentiment(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=1).item()
    return "Positive 😊" if pred == 1 else "Negative 😠"

# Example
while True:
    user_input = input("Enter a review (or type 'exit'): ")
    if user_input.lower() == "exit":
        break
    result = predict_sentiment(user_input)
    print("Prediction:", result)

Enter a review (or type 'exit'):  i hate it


Prediction: Negative 😠


Enter a review (or type 'exit'):  good but bad


Prediction: Negative 😠


Enter a review (or type 'exit'):  good product


...

Enter a review (or type 'exit'):  I am writing to formally complain about my recent purchase of an iPhone 16 from online on Amazon. I received the item on the same day of purchase, but upon unboxing, I noticed that the iPhone came in a mismatched (muddled) box. When I powered it on, the phone was stuck on the “Hello” screen, repeatedly displaying “Hello” in multiple languages. It was completely unresponsive—none of the buttons, including the power and volume buttons, were functioning.  After approximately five hours, the battery finally drained, and the phone shut down. However, after restarting, the same issue persisted, with the device stuck on the “Hello” screen again.  I contacted Amazon’s support team, who advised me to reach out to Apple Support. Apple Support suggested some troubleshooting steps, but none resolved the issue. Ultimately, they advised me to visit an Apple-authorized service center for a software reinstallation—on a brand-new phone.  I then had to travel 5 km to a 

Prediction: Negative 😠


Enter a review (or type 'exit'):  Very light and better than previous versions. Or let’s say everything is great but I’m little disappointed by the camera, I believe the best camera they had was with 11, after all my upgrades and finally to 16, I didn’t find such stable and best in class camera. AI in iOS is still evolving and not yet that great.


Prediction: Positive 😊


Enter a review (or type 'exit'):  exit


In [6]:
torch.cuda.empty_cache()